# Обучение с подкреплением

***Второй этап, обучение на победу***

**Выполнил Кузин Мирослав**

In [15]:
from mongala import Kalah
import matplotlib.pyplot as plt
%matplotlib inline
from IPython import display
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical
import numpy as np
import pandas as pd
import time
import random

**Болваны**

In [16]:
# Бот простой, рандомно выбирает не нулевые элементы
def do_simple_bot_step(state: torch.Tensor) -> int:
    nonzero_state_indexs = torch.nonzero(state[:6]).flatten()
    rez = nonzero_state_indexs[torch.randint(0, len(nonzero_state_indexs), (1,))[0]]
    return rez + 1

do_simple_bot_step(torch.Tensor([1, 0, 1, 0, 0, 0, 0]))

tensor(1)

**Модель**

In [17]:
model = torch.load("./model_non_zero_top_tryed3.pt")

**Настройка обучения**

In [18]:
def loss_func(probs: torch.Tensor, action: int, m: Categorical, R: int, gamma: int):
    alpha = 0.9
    # print(Categorical(probs).log_prob(action))
    return -alpha*R*m.log_prob(action)

loss = loss_func
optimizer = torch.optim.Adam(model.parameters(), lr=1.0e-4, amsgrad=True)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


**Обучение**

In [19]:
# Проверка работы модели
game = Kalah()
model.to(device)
model.to(torch.double)

print(game.get_general_state())
model(game.get_general_state().to(torch.double))

tensor([6, 6, 6, 6, 6, 6, 0, 0, 6, 6, 6, 6, 6, 6])


tensor([1.6487e-03, 9.9037e-01, 7.9768e-03, 3.3244e-06, 3.4061e-06, 7.6303e-08],
       dtype=torch.float64, grad_fn=<SoftmaxBackward0>)

*Награды и штрафы*

In [71]:
# Второй этап обучения
winner_reward_stage_2 = 20
loser_reward_stage_2 = -20
draw_reward_stage_2 = -10
bad_step_reward_stage_2 = -30 # общий
good_step_reward_stage_2 = 1
good_step_captured_reward_stage_2 = 2

In [72]:
game = Kalah()
episodes_count = 10000
neuronet_walker_queue = 1
count_win_neuronet = 0
count_choisen_zero = 0
gamma = 2
loss_story = []
check_optim = False

for episode in range(0, episodes_count):
    game.set_new_game()
    tr = []
    if episode % 2:
        old_player_making_step = game.get_player_making_step()
        neuronet_walker_queue = 2
        while not game.get_game_over() and old_player_making_step == game.get_player_making_step():
            bot_action = do_simple_bot_step(game.get_state())
            rezult_step = game.take_step(bot_action)
            # print("bot took step", game.get_player_making_step(), "Был выбор", bot_action, rezult_step)
    else:
        neuronet_walker_queue = 1

    while not game.get_game_over():

        # Выбор хода
        probs = model(game.get_state().to(torch.double))
        m = Categorical(probs)
        action = m.sample()

        # Выбор награды и выполнение хода
        reward = 0
        old_state = game.get_state().to(torch.double)
        old_player_making_step = game.get_player_making_step()

        rezult_step = game.take_step(action + 1)
        # print("neuronet took step", game.get_player_making_step(), "Был выбор", action + 1, rezult_step)
        if rezult_step == "Куча пустая, выберите другую!":
            count_choisen_zero += 1
            reward = bad_step_reward_stage_2
        elif rezult_step == "Хороший ход!":
            reward = good_step_reward_stage_2
            bad_choise = []
        elif rezult_step == "Хороший ход! Захват!":
            reward = good_step_captured_reward_stage_2
            bad_choise = []

        tr.append([old_state, action, reward, m])

        while not game.get_game_over() and old_player_making_step != game.get_player_making_step():
            bot_action = do_simple_bot_step(game.get_state())
            rezult_step = game.take_step(bot_action)
            # print("bot took step", game.get_player_making_step(), "Был выбор", bot_action, rezult_step)

    rwrd = 0
    if game.get_winner() != None:
        if game.get_winner() != 0:
            if neuronet_walker_queue == game.get_winner():
                count_win_neuronet += 1
                rwrd += winner_reward_stage_2
            else:
                rwrd += loser_reward_stage_2
        else:
            # tr[-1][-2] += draw_reward_stage_2 if tr[-1][-2] > 0 else 0
            rwrd += draw_reward_stage_2

    loss = 0.
    count_played_step = len(tr) # Кол-во сыгранных ходов
    for id_current_step in range(count_played_step):
        R = 0.
        for id_next_step in range(id_current_step, count_played_step):
            R += (gamma**(id_current_step-id_next_step))*tr[id_next_step][2] + rwrd
        loss += loss_func(model(tr[id_current_step][0]), tr[id_current_step][1], tr[id_current_step][3], R, gamma)

    if not check_optim and episode > 10000:
        check_optim = True
        for g in optimizer.param_groups:
            g['lr'] = 1.0e-5
        print("updated loss")

    if not episode % 200:
        print(loss)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

tensor(313.5733, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(-6923.2118, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(1772.0152, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(-879.5154, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(-3687.4191, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(-1815.1157, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(-247.3717, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(943.0535, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(1241.0074, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(3172.2925, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(1491.9932, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(-1485.2762, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(3160.8632, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(1433.9496, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(756.9171, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(-121.8381, dtype=torch.float64, grad_fn=<AddBac

In [73]:
print(check_optim)
print(count_choisen_zero)

False
3253


**Тестирование модели**

In [77]:
game = Kalah()
episodes_count = 1000
neuronet_walker_queue = 1
count_win_neuronet = 0
count_choisen_zero = 0
bad_choise = []
rezult_step = "Хороший ход!"

for episode in range(episodes_count):
    game.set_new_game()
    if episode % 2:
        old_player_making_step = game.get_player_making_step()
        neuronet_walker_queue = 2
        while not game.get_game_over() and old_player_making_step == game.get_player_making_step():
            bot_action = do_simple_bot_step(game.get_state())
            rezult_step = game.take_step(bot_action)
            # print("bot took step", game.get_player_making_step(), "Был выбор", bot_action, rezult_step)
    else:
        neuronet_walker_queue = 1

    while not game.get_game_over():

        # Выбор хода
        probs = model(game.get_state().to(torch.double))
        action = probs.argmax()

        while action in bad_choise:
            probs[probs.argmax()] = 0
            action = probs.argmax()

        # Выбор награды и выполнение хода
        reward = 0
        old_state = game.get_state().to(torch.double)
        old_player_making_step = game.get_player_making_step()

        rezult_step = game.take_step(action + 1)
        if rezult_step == "Куча пустая, выберите другую!":
            count_choisen_zero += 1
            # reward = bad_step_reward_stage_1
            print("Neuronet took step", game.get_player_making_step(), "Был выбор", action + 1, rezult_step)
            # print("state:")
            # game.print_state()
            print("Номер эпизода происшествия:", episode)
            bad_choise.append(action)
        else:
            bad_choise = []
        # elif rezult_step == "Хороший ход!":
        #     reward = good_step_reward_stage_1
        # elif rezult_step == "Хороший ход! Захват!":
        #     reward = good_step_captured_reward_stage_1

        while not game.get_game_over() and old_player_making_step != game.get_player_making_step():
            bot_action = do_simple_bot_step(game.get_state())
            rezult_step = game.take_step(bot_action)
            if rezult_step == "Куча пустая, выберите другую!":
                print("Bot took step", game.get_player_making_step(), "Был выбор", bot_action, rezult_step)
            # print("bot took step", game.get_player_making_step(), "Был выбор", bot_action, rezult_step)


    if game.get_winner() != None:
        if game.get_winner() != 0:
            if neuronet_walker_queue == game.get_winner():
                count_win_neuronet += 1
                reward += winner_reward_stage_2
            else:
                reward += loser_reward_stage_2
        else:
            reward += draw_reward_stage_2

    # print("Очередь хода нейронки", neuronet_walker_queue)
    # print("Результат:", game.get_player_winner(), 'k', game.get_winner())
    # print("Номер попытки:", episode)
    # print("Награда/Штраф:", reward)

    # print(episode)

print(count_choisen_zero)
print(count_win_neuronet/episodes_count)

0
0.748


In [75]:
is_save = True
# is_save = False
if is_save:
    torch.save(model, "./model_winner_ver4.pt")